In [1]:
import torch
import numpy as np
import random

device = 'cuda:0' if torch.cuda.is_available else 'cpu'

In [2]:
from auto_LiRPA import BoundedModule, PerturbationLpNorm, BoundedTensor
from auto_LiRPA.utils import get_spec_matrix
from cert_util import min_correct_with_eps, load_data, DeltaWrapper

from model_defs import mnist_cnn_4layer,mnist_conv_small,mnist_conv_big

In [3]:
def bounded_results(eps,bounded_model):
    ptb = PerturbationLpNorm(norm = np.inf, eps = eps)
    bounded_delta = BoundedTensor(delta, ptb)

    final_name = bounded_model.final_name
    input_name = '/1' # '/input.1' 

    result = bounded_model.compute_bounds(
        x=(new_image,bounded_delta), method='backward', C=C,
        return_A=True, 
        needed_A_dict={ final_name: [input_name] },
    )
    lower, upper, A_dict = result
    lA = A_dict[final_name][input_name]['lA']
    uA = A_dict[final_name][input_name]['uA']

    lb = lower - ptb.concretize(delta, lA, sign=-1)
    ub = upper - ptb.concretize(delta, uA, sign=1)


    lA = torch.reshape(lA,(eval_num, num_cls-1,-1))
    return lA,lb,lower

### Reproducibility

In [4]:
my_seed = 1
torch.cuda.empty_cache()
torch.manual_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

### Hyperparameters 

In [5]:
eval_num = 100
num_cls = 10
adv_e = 8

# MNIST models, one can choose from 'mnist_conv_small', 'mnist_cnn_4layer', 'mnist_conv_big'
model_name = "mnist_conv_small" 

### Loadding the model for certification

In [6]:
net = eval(model_name)()
net.load_state_dict(torch.load('./model_weights/'+model_name+'.pth'))

<All keys matched successfully>

### Loading a batch of data

In [7]:
new_image, new_label = load_data(num_imgs=eval_num, random=True, dataset='MNIST')
new_image = new_image.to(device)
C = get_spec_matrix(new_image,new_label.long(),10)

### Model concretization

In [8]:
eps = adv_e/255
delta = torch.zeros_like(new_image[0]).unsqueeze(0)
dummy_input = (new_image, delta)
model = DeltaWrapper(net.to(device))
bounded_model = BoundedModule(model, dummy_input)
bounded_model.eval()
final_name = bounded_model.final_name

### Results and comparision 

In [9]:
print('Eps:', adv_e)

alpha,beta,result = bounded_results(eps,bounded_model)
samp_ACC = torch.sum(result.detach().cpu().min(axis=1)[0] > 0).numpy()
print('Samp-wise Cert-ACC: {}%'.format(samp_ACC))

label = new_label
number_class = num_cls
cert_ACC, delta = min_correct_with_eps(alpha, beta, eps, label, number_class=10, verbose=False)
print('UP-based Cert-ACC: {}%'.format(cert_ACC))

Eps: 8
Samp-wise Cert-ACC: 50%
Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-24
Set parameter TimeLimit to value 600
UP-based Cert-ACC: 66.0%
